<a href="https://colab.research.google.com/github/anushkagupta1307/IR2022_Project_27/blob/main/IR_Project_SecondBaseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Method to get Reviews from a product on Amazon-
#This method takes the Product URL as the input and fetches all the Reviews on the product on that URL
#Implemented using BeautifulSoup library
#The method returns a dataframe consisting of Customer Name, Review Title, Rating and Review
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
def scraping_amazon_reviews(url):

  flag=0;
  count=0;
  u=1;

  link=url
  page = requests.get(link)
  soup = bs(page.content,'html.parser')
  total_ratings = soup.find_all("",{"data-hook":"cr-filter-info-review-rating-count"})
  print(total_ratings)
  total_ratings_lstrips=total_ratings[0].get_text().lstrip('\n')
  total_ratings_rstrips=total_ratings_lstrips.rstrip('\n')
  total_ratings_rstrips = total_ratings_rstrips.replace(",", "")
  result = total_ratings_rstrips.index('total ratings')

  position=0
  for i in range(result-2,0,-1):
      if total_ratings_rstrips[i]==' ':
        position=i
        break;

  total_ratings_final=int(total_ratings_rstrips[position+1:result-1])
  
  df = pd.DataFrame(columns = ['Customer Name', 'Review title', 'Ratings','Reviews'])
  # u<=(total_ratings_final/10)
  while(u<=(300)):
    url=url+"&pageNumber="+str(u)
    page = requests.get(url)

    if page.status_code!=200:
      #print("breaked at",u)
      flag=1
      continue;
    
    soup = bs(page.content,'html.parser')

    names = soup.find_all('span',class_='a-profile-name')
    cust_name = []
    for i in range(0,len(names)):
        cust_name.append(names[i].get_text())

    cust_name.pop(0)
    cust_name.pop(0)

    indexes_to_remove=[]
    for i in range(len(cust_name)):
      
      for j in range(i+1,len(cust_name)):
        if cust_name[i]==cust_name[j]:
          #print(cust_name[i])
          indexes_to_remove.append(j)

    for i in range(len(indexes_to_remove)-1,-1,-1):
        cust_name.pop(indexes_to_remove[i])


    title = soup.find_all('a',class_='review-title-content')
    review_title = []
    for i in range(0,len(title)):
        review_title.append(title[i].get_text())
    
    review_title[:] = [titles.lstrip('\n') for titles in review_title]
    review_title[:] = [titles.rstrip('\n') for titles in review_title]

    
    rating = soup.find_all('i',class_='review-rating')
    rate = []
    for i in range(0,len(rating)):
        rate.append(rating[i].get_text())

    rate.pop(0)
    rate.pop(0)

    review = soup.find_all("span",{"data-hook":"review-body"})

    review_content = []
    for i in range(0,len(review)):
        review_content.append(review[i].get_text())

    review_content[:] = [reviews.lstrip('\n') for reviews in review_content]
    review_content[:] = [reviews.rstrip('\n') for reviews in review_content]
    #print(review_content)

    for i in range(len(cust_name)):
      
      df=df.append({'Customer Name':cust_name[i], 'Review title':review_title[i], 'Ratings':rate[i],'Reviews':review_content[i]},ignore_index=True)

    u=u+1
    count=count+1

  #print(df)
  return df;


In [ ]:
#Calling the above defined method with a Redmi Mobile Amazon URL
df=scraping_amazon_reviews("https://www.amazon.in/Redmi-Activ-Metallic-Purple-Storage/product-reviews/B09GFNZT24/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews")

[<div class="a-row a-spacing-base a-size-base" data-hook="cr-filter-info-review-rating-count">



















    
        
            
                    
                        1,52,180 total ratings, 14,093 with reviews
                    
            
            
        
    
    


</div>]


In [ ]:
#Dispalying the generated dataframe
df

,Customer Name,Review title,Ratings,Reviews
0,Ashish,DONT BUY THIS MOBILE .,1.0 out of 5 stars,Don’t buy this product .Image quality is below...
1,ADIL.T.M,"Redmi 9 hhoo, 9, 9🤩, 9, changed mind",4.0 out of 5 stars,Let’s make one thing clear immediately: the Xi...
2,Taufeeque Alam,Awesome,5.0 out of 5 stars,Awesome
3,Amazon Customer,Worst experience,1.0 out of 5 stars,Very bad experience from Amazon. Very bad came...
4,Sourabh Meena,Best in budget,5.0 out of 5 stars,Best buy at this price! The display quality is...
...,...,...,...,...
2395,Amazon Customer,Worst experience,1.0 out of 5 stars,Very bad experience from Amazon. Very bad came...
2396,Sourabh Meena,Best in budget,5.0 out of 5 stars,Best buy at this price! The display quality is...
2397,Thirunavukarasu,Think it before buy,4.0 out of 5 stars,i didn't upload photo ....for any reason but i...
2398,Anup mall,Everything is fine except camera which only wo...,1.0 out of 5 stars,This phone is not value for moneyIts a downgra...


In [ ]:
#Extract the ratings as a float value from the above Rating column and create a separate column in the dataframe for float rating
extracted_ratings=[]
for i in range(len(df)):
  for j in range(len(df['Ratings'][i])):
    if df['Ratings'][i][j]==" ":
      count=j
      break;
  #print(count)

  #print(df_sorted_price['overall rating'][i][:count])
  extracted_ratings.append(float(df['Ratings'][i][:count]))


In [ ]:
df['extracted_ratings']=extracted_ratings

In [ ]:
df

,Customer Name,Review title,Ratings,Reviews,extracted_ratings
0,Ashish,DONT BUY THIS MOBILE .,1.0 out of 5 stars,Don’t buy this product .Image quality is below...,1.0
1,ADIL.T.M,"Redmi 9 hhoo, 9, 9🤩, 9, changed mind",4.0 out of 5 stars,Let’s make one thing clear immediately: the Xi...,4.0
2,Taufeeque Alam,Awesome,5.0 out of 5 stars,Awesome,5.0
3,Amazon Customer,Worst experience,1.0 out of 5 stars,Very bad experience from Amazon. Very bad came...,1.0
4,Sourabh Meena,Best in budget,5.0 out of 5 stars,Best buy at this price! The display quality is...,5.0
...,...,...,...,...,...
2395,Amazon Customer,Worst experience,1.0 out of 5 stars,Very bad experience from Amazon. Very bad came...,1.0
2396,Sourabh Meena,Best in budget,5.0 out of 5 stars,Best buy at this price! The display quality is...,5.0
2397,Thirunavukarasu,Think it before buy,4.0 out of 5 stars,i didn't upload photo ....for any reason but i...,4.0
2398,Anup mall,Everything is fine except camera which only wo...,1.0 out of 5 stars,This phone is not value for moneyIts a downgra...,1.0


In [ ]:
#FIRST METHODOLOGY- Give a score of 1 if the rating on a comment is 4 or 5, give a score 0 if the rating is 3
# and give a score of -1 if the rating on the comment is 1 or 2
rating_score=[]
for i in range(len(df)):
  if df['extracted_ratings'][i]>=4:
    rating_score.append(1)
  elif df['extracted_ratings'][i]==3:
    rating_score.append(0)
  else:
    rating_score.append(-1)




In [ ]:
#Append the rating score column on the dataframe
df['rating_score']=rating_score

In [ ]:
df

,Customer Name,Review title,Ratings,Reviews,extracted_ratings,rating_score
0,Ashish,DONT BUY THIS MOBILE .,1.0 out of 5 stars,Don’t buy this product .Image quality is below...,1.0,-1
1,ADIL.T.M,"Redmi 9 hhoo, 9, 9🤩, 9, changed mind",4.0 out of 5 stars,Let’s make one thing clear immediately: the Xi...,4.0,1
2,Taufeeque Alam,Awesome,5.0 out of 5 stars,Awesome,5.0,1
3,Amazon Customer,Worst experience,1.0 out of 5 stars,Very bad experience from Amazon. Very bad came...,1.0,-1
4,Sourabh Meena,Best in budget,5.0 out of 5 stars,Best buy at this price! The display quality is...,5.0,1
...,...,...,...,...,...,...
2395,Amazon Customer,Worst experience,1.0 out of 5 stars,Very bad experience from Amazon. Very bad came...,1.0,-1
2396,Sourabh Meena,Best in budget,5.0 out of 5 stars,Best buy at this price! The display quality is...,5.0,1
2397,Thirunavukarasu,Think it before buy,4.0 out of 5 stars,i didn't upload photo ....for any reason but i...,4.0,1
2398,Anup mall,Everything is fine except camera which only wo...,1.0 out of 5 stars,This phone is not value for moneyIts a downgra...,1.0,-1


In [ ]:
#Method to generate the polarity score of a text
from textblob import TextBlob
# function to calculate polarity
def getPolarity(review):
  return TextBlob(review).sentiment.polarity

In [ ]:
#Run a loop on all reviews in the dataframe and generate the polarity score of each review
polarity_scores_for_df=[]
for i in range(len(df)):
  temp_polarity=float(getPolarity(df['Reviews'][i]))
  polarity_scores_for_df.append(temp_polarity)


In [ ]:
#Append the polarity score of the review in the dataframe
df['polarity_scores']=polarity_scores_for_df

In [ ]:
df

,Customer Name,Review title,Ratings,Reviews,extracted_ratings,rating_score,polarity_scores
0,Ashish,DONT BUY THIS MOBILE .,1.0 out of 5 stars,Don’t buy this product .Image quality is below...,1.0,-1,-0.162500
1,ADIL.T.M,"Redmi 9 hhoo, 9, 9🤩, 9, changed mind",4.0 out of 5 stars,Let’s make one thing clear immediately: the Xi...,4.0,1,0.175366
2,Taufeeque Alam,Awesome,5.0 out of 5 stars,Awesome,5.0,1,1.000000
3,Amazon Customer,Worst experience,1.0 out of 5 stars,Very bad experience from Amazon. Very bad came...,1.0,-1,-0.455000
4,Sourabh Meena,Best in budget,5.0 out of 5 stars,Best buy at this price! The display quality is...,5.0,1,0.352500
...,...,...,...,...,...,...,...
2395,Amazon Customer,Worst experience,1.0 out of 5 stars,Very bad experience from Amazon. Very bad came...,1.0,-1,-0.455000
2396,Sourabh Meena,Best in budget,5.0 out of 5 stars,Best buy at this price! The display quality is...,5.0,1,0.352500
2397,Thirunavukarasu,Think it before buy,4.0 out of 5 stars,i didn't upload photo ....for any reason but i...,4.0,1,0.116667
2398,Anup mall,Everything is fine except camera which only wo...,1.0 out of 5 stars,This phone is not value for moneyIts a downgra...,1.0,-1,-0.075769


In [ ]:
df.columns

Index(['Customer Name', 'Review title', 'Ratings', 'Reviews',
       'extracted_ratings', 'rating_score', 'polarity_scores'],
      dtype='object')

In [ ]:
#Classification of reviews based on polarity scores-
#If the polarity is >0 then class 1 to polarity score
#If the polarity is <0 , -1 is the polarity score
#else for polarity=0 , polarity score is 0.
polarity_score_extracted=[]
for i in range(len(df)):
  if df['polarity_scores'][i]>0:
    polarity_score_extracted.append(1)
  elif df['polarity_scores'][i]==0:
    polarity_score_extracted.append(0)
  else:
    polarity_score_extracted.append(-1)



In [ ]:
#append the polarity score column to the dataframe
df['polarity_score_extracted']=polarity_score_extracted

In [ ]:
df

,Customer Name,Review title,Ratings,Reviews,extracted_ratings,rating_score,polarity_scores,polarity_score_extracted
0,Ashish,DONT BUY THIS MOBILE .,1.0 out of 5 stars,Don’t buy this product .Image quality is below...,1.0,-1,-0.162500,-1
1,ADIL.T.M,"Redmi 9 hhoo, 9, 9🤩, 9, changed mind",4.0 out of 5 stars,Let’s make one thing clear immediately: the Xi...,4.0,1,0.175366,1
2,Taufeeque Alam,Awesome,5.0 out of 5 stars,Awesome,5.0,1,1.000000,1
3,Amazon Customer,Worst experience,1.0 out of 5 stars,Very bad experience from Amazon. Very bad came...,1.0,-1,-0.455000,-1
4,Sourabh Meena,Best in budget,5.0 out of 5 stars,Best buy at this price! The display quality is...,5.0,1,0.352500,1
...,...,...,...,...,...,...,...,...
2395,Amazon Customer,Worst experience,1.0 out of 5 stars,Very bad experience from Amazon. Very bad came...,1.0,-1,-0.455000,-1
2396,Sourabh Meena,Best in budget,5.0 out of 5 stars,Best buy at this price! The display quality is...,5.0,1,0.352500,1
2397,Thirunavukarasu,Think it before buy,4.0 out of 5 stars,i didn't upload photo ....for any reason but i...,4.0,1,0.116667,1
2398,Anup mall,Everything is fine except camera which only wo...,1.0 out of 5 stars,This phone is not value for moneyIts a downgra...,1.0,-1,-0.075769,-1


In [ ]:
#Extract all the reviews which have the word "NOT" in them
#Get all the words after the word "NOT"
#Put all the words after "NOT" in a string 
reviews_extracted_for_not=[]
for i in range(len(df)):
  if "not" in df['Reviews'][i].lower():
    words = df['Reviews'][i].lower().split(' ')
    for j in range(len(words)):
      if words[j]=="not":
        start=j
        break;
    
    temp_str=""
    for j in range(start+1,len(words),1):
      temp_str=temp_str+" "+words[j]
    reviews_extracted_for_not.append(temp_str)
  
  else:
    temp_str=""
    reviews_extracted_for_not.append(temp_str)


In [ ]:
#import necessary libraries
import os
import pydrive
import re
from nltk.corpus import stopwords
from gensim.parsing.preprocessing import remove_stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
ps = PorterStemmer()
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import nltk
#nltk library for preprocessing
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
#Preprocessing on the string which contains all the words after "NOT"
#remove whitespaces, Remove numerics, Remove special characters, Remove Stopwords, convert to lowercase
preprocessed_etxracted_list_for_not=[]
for i in range(len(reviews_extracted_for_not)):
  temp_review=reviews_extracted_for_not[i]
  temp_review = temp_review.lower()
  temp_review = re.sub(r'\d+','',temp_review)

  temp_review = re.sub(r'[^\w\s]','', temp_review)
  temp_review= re.sub('[@_!#$%^&*()<>?/\|}{~:]','',temp_review)
  temp_review = temp_review.strip()
  temp_review = remove_stopwords(temp_review)
  words = word_tokenize(temp_review)
  temp_string=""
  for word in words:
    temp_string=temp_string+" "+(word)

  preprocessed_etxracted_list_for_not.append(temp_string)

In [ ]:
from textblob import TextBlob
# function to calculate polarity
def getPolarity(review):
  return TextBlob(review).sentiment.polarity

In [ ]:
#Calculate the Polarity score of all the words in the string and get the sum of the polarity score
not_polarity_score=[]
for i in range(len(preprocessed_etxracted_list_for_not)):
  words=preprocessed_etxracted_list_for_not[i].split(' ')
  words_polarity=0.0
  for word in words:
    if word=="":
      polarity=0.0
    else:
      polarity=getPolarity(word)
    
    words_polarity=words_polarity+float(polarity)

  not_polarity_score.append(words_polarity)
    

In [ ]:
len(not_polarity_score)

2400

In [ ]:
#If the sum of polarity score of all words in the string is >0- classify it as 1, 
#if <0 then classify it as -1, if 0 then classify it as 0
score_based_on_polarity=[]
for i in range(len(not_polarity_score)):
  if not_polarity_score[i]>0:
    temp_polarity_score=-1
  elif not_polarity_score[i]==0:
    temp_polarity_score=0
  else:
    temp_polarity_score=1

  score_based_on_polarity.append(temp_polarity_score)


In [ ]:
#Append this polarity score after the NOT word in the dataframe
df["polarity score on not query"]=score_based_on_polarity

In [ ]:
df

,Customer Name,Review title,Ratings,Reviews,extracted_ratings,rating_score,polarity_scores,polarity_score_extracted,polarity score on not query
0,Ashish,DONT BUY THIS MOBILE .,1.0 out of 5 stars,Don’t buy this product .Image quality is below...,1.0,-1,-0.162500,-1,0
1,ADIL.T.M,"Redmi 9 hhoo, 9, 9🤩, 9, changed mind",4.0 out of 5 stars,Let’s make one thing clear immediately: the Xi...,4.0,1,0.175366,1,-1
2,Taufeeque Alam,Awesome,5.0 out of 5 stars,Awesome,5.0,1,1.000000,1,0
3,Amazon Customer,Worst experience,1.0 out of 5 stars,Very bad experience from Amazon. Very bad came...,1.0,-1,-0.455000,-1,0
4,Sourabh Meena,Best in budget,5.0 out of 5 stars,Best buy at this price! The display quality is...,5.0,1,0.352500,1,-1
...,...,...,...,...,...,...,...,...,...
2395,Amazon Customer,Worst experience,1.0 out of 5 stars,Very bad experience from Amazon. Very bad came...,1.0,-1,-0.455000,-1,0
2396,Sourabh Meena,Best in budget,5.0 out of 5 stars,Best buy at this price! The display quality is...,5.0,1,0.352500,1,-1
2397,Thirunavukarasu,Think it before buy,4.0 out of 5 stars,i didn't upload photo ....for any reason but i...,4.0,1,0.116667,1,-1
2398,Anup mall,Everything is fine except camera which only wo...,1.0 out of 5 stars,This phone is not value for moneyIts a downgra...,1.0,-1,-0.075769,-1,1


In [ ]:
##Preprocessing on the whole review
preprocessed_extracted_list_of_word=[]
for i in range(len(df)):
  temp_review=df['Reviews'][i]
  temp_review = temp_review.lower()
  temp_review = re.sub(r'\d+','',temp_review)

  temp_review = re.sub(r'[^\w\s]','', temp_review)
  temp_review= re.sub('[@_!#$%^&*()<>?/\|}{~:]','',temp_review)
  temp_review = temp_review.strip()
  temp_review = remove_stopwords(temp_review)
  words = word_tokenize(temp_review)
  temp_string=""
  for word in words:
    temp_string=temp_string+" "+(word)

  preprocessed_extracted_list_of_word.append(temp_string)

In [ ]:
#get the polarity score of all words in the review after preprocessing
#Get the sum of polarity scores
total_words_polarity_score=[]
for i in range(len(preprocessed_extracted_list_of_word)):
  words=preprocessed_extracted_list_of_word[i].split(' ')

  words_polarity=0.0
  for word in words:
    if word=="":
      polarity=0.0
    else:
      polarity=getPolarity(word)
    
    words_polarity=words_polarity+float(polarity)

  total_words_polarity_score.append(words_polarity)

  

In [ ]:
total_words_polarity_score

[-0.65,
 10.140476190476184,
 1.0,
 -1.3999999999999997,
 1.7000000000000002,
 1.4,
 -0.44999999999999973,
 0.19999999999999996,
 -0.65,
 10.140476190476184,
 1.0,
 -1.3999999999999997,
 1.7000000000000002,
 1.4,
 -0.44999999999999973,
 0.19999999999999996,
 -0.65,
 10.140476190476184,
 1.0,
 -1.3999999999999997,
 1.7000000000000002,
 1.4,
 -0.44999999999999973,
 0.19999999999999996,
 -0.65,
 10.140476190476184,
 1.0,
 -1.3999999999999997,
 1.7000000000000002,
 1.4,
 -0.44999999999999973,
 0.19999999999999996,
 -0.65,
 10.140476190476184,
 1.0,
 -1.3999999999999997,
 1.7000000000000002,
 1.4,
 -0.44999999999999973,
 0.19999999999999996,
 -0.65,
 10.140476190476184,
 1.0,
 -1.3999999999999997,
 1.7000000000000002,
 1.4,
 -0.44999999999999973,
 0.19999999999999996,
 -0.65,
 10.140476190476184,
 1.0,
 -1.3999999999999997,
 1.7000000000000002,
 1.4,
 -0.44999999999999973,
 0.19999999999999996,
 -0.65,
 10.140476190476184,
 1.0,
 -1.3999999999999997,
 1.7000000000000002,
 1.4,
 -0.449999999

In [ ]:
#If the sum of polarity score of all words in the string is >0- classify it as 1, 
#if <0 then classify it as -1, if 0 then classify it as 0
total_words_polarity_value=[]
for i in range(len(total_words_polarity_score)):
  if total_words_polarity_score[i]>0:
    total_words_polarity_value.append(1)
  elif total_words_polarity_score[i]==0:
    total_words_polarity_value.append(0)
  else:
    total_words_polarity_value.append(-1)



In [ ]:
#Append the column which consists of polarity score of whole review after preprocessing
df['total_words_polarity_value']=total_words_polarity_value

In [ ]:
df

,Customer Name,Review title,Ratings,Reviews,extracted_ratings,rating_score,polarity_scores,polarity_score_extracted,polarity score on not query,total_words_polarity_value
0,Ashish,DONT BUY THIS MOBILE .,1.0 out of 5 stars,Don’t buy this product .Image quality is below...,1.0,-1,-0.162500,-1,0,-1
1,ADIL.T.M,"Redmi 9 hhoo, 9, 9🤩, 9, changed mind",4.0 out of 5 stars,Let’s make one thing clear immediately: the Xi...,4.0,1,0.175366,1,-1,1
2,Taufeeque Alam,Awesome,5.0 out of 5 stars,Awesome,5.0,1,1.000000,1,0,1
3,Amazon Customer,Worst experience,1.0 out of 5 stars,Very bad experience from Amazon. Very bad came...,1.0,-1,-0.455000,-1,0,-1
4,Sourabh Meena,Best in budget,5.0 out of 5 stars,Best buy at this price! The display quality is...,5.0,1,0.352500,1,-1,1
...,...,...,...,...,...,...,...,...,...,...
2395,Amazon Customer,Worst experience,1.0 out of 5 stars,Very bad experience from Amazon. Very bad came...,1.0,-1,-0.455000,-1,0,-1
2396,Sourabh Meena,Best in budget,5.0 out of 5 stars,Best buy at this price! The display quality is...,5.0,1,0.352500,1,-1,1
2397,Thirunavukarasu,Think it before buy,4.0 out of 5 stars,i didn't upload photo ....for any reason but i...,4.0,1,0.116667,1,-1,1
2398,Anup mall,Everything is fine except camera which only wo...,1.0 out of 5 stars,This phone is not value for moneyIts a downgra...,1.0,-1,-0.075769,-1,1,-1


In [ ]:
df.columns

Index(['Customer Name', 'Review title', 'Ratings', 'Reviews',
       'extracted_ratings', 'rating_score', 'polarity_scores',
       'polarity_score_extracted', 'polarity score on not query',
       'total_words_polarity_value'],
      dtype='object')

In [ ]:
#get all the four types of polarity scores
for i in range(len(df)):
  df['rating_score'][i]=float(df['rating_score'][i])
  df['polarity_score_extracted'][i]=float(df['polarity_score_extracted'][i])
  df['polarity score on not query'][i]=float(df['polarity score on not query'][i])
  df['total_words_polarity_value'][i]=float(df['total_words_polarity_value'][i])
  



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from 

In [ ]:
#Give equal weightage to all the types of polarity scores and then calculate the average polarity score based on which the review will be classified
total_score=[]
for i in range(len(df)):
  temp_score=0.25*df['rating_score'][i]+0.25*df['polarity_score_extracted'][i]+0.25*df['polarity score on not query'][i]+0.25*df['total_words_polarity_value'][i]
  total_score.append(temp_score)


In [ ]:
#Append the total score column to the dataframe
df['total score']=total_score

In [ ]:
df

,Customer Name,Review title,Ratings,Reviews,extracted_ratings,rating_score,polarity_scores,polarity_score_extracted,polarity score on not query,total_words_polarity_value,total score
0,Ashish,DONT BUY THIS MOBILE .,1.0 out of 5 stars,Don’t buy this product .Image quality is below...,1.0,-1,-0.162500,-1,0,-1,-0.75
1,ADIL.T.M,"Redmi 9 hhoo, 9, 9🤩, 9, changed mind",4.0 out of 5 stars,Let’s make one thing clear immediately: the Xi...,4.0,1,0.175366,1,-1,1,0.50
2,Taufeeque Alam,Awesome,5.0 out of 5 stars,Awesome,5.0,1,1.000000,1,0,1,0.75
3,Amazon Customer,Worst experience,1.0 out of 5 stars,Very bad experience from Amazon. Very bad came...,1.0,-1,-0.455000,-1,0,-1,-0.75
4,Sourabh Meena,Best in budget,5.0 out of 5 stars,Best buy at this price! The display quality is...,5.0,1,0.352500,1,-1,1,0.50
...,...,...,...,...,...,...,...,...,...,...,...
2395,Amazon Customer,Worst experience,1.0 out of 5 stars,Very bad experience from Amazon. Very bad came...,1.0,-1,-0.455000,-1,0,-1,-0.75
2396,Sourabh Meena,Best in budget,5.0 out of 5 stars,Best buy at this price! The display quality is...,5.0,1,0.352500,1,-1,1,0.50
2397,Thirunavukarasu,Think it before buy,4.0 out of 5 stars,i didn't upload photo ....for any reason but i...,4.0,1,0.116667,1,-1,1,0.50
2398,Anup mall,Everything is fine except camera which only wo...,1.0 out of 5 stars,This phone is not value for moneyIts a downgra...,1.0,-1,-0.075769,-1,1,-1,-0.50


In [ ]:
#If the total score polarity > 0, then the review is positive
#If the total score polarity < 0, then the review is negative
#If the total score polarity = 0, then the review is nautral
total_score_polarity=[]
for i in range(len(df)):
  if df['total score'][i]>0:
    total_score_polarity.append(1)
  elif df['total score'][i]==0:
    total_score_polarity.append(0)
  else:
    total_score_polarity.append(-1)



In [ ]:
df['total_score_polarity']=total_score_polarity

In [ ]:
#Count the total number of positive and negative reviews on the prodcut
countpositive=0
countnegative=0

for i in range(len(df)):
  if df['total_score_polarity'][i]==1:
    countpositive=countpositive+1
  else:
    countnegative=countnegative+1

print(countpositive)
print(countnegative)

1500
900


In [ ]:
if(countpositive>countnegative):
  print("You can go for the selected device :) ")
else:
  print("Maybe you can go with any other option available :( ")

You can go for the selected device :) 
